## Spark + Scala + Cassandra examples

Imports! The datastax jars are baked into my Dockerfile.notebook file, this is a slight pain, you need the scala version to match with those on the spark workers

In [9]:
import scala.math.random                                                                   
import org.apache.spark.SparkContext                                                       
import org.apache.spark.SparkContext._                                                     
import org.apache.spark.SparkConf                                                          
import org.apache.spark.sql.SparkSession                                                   
import com.datastax.spark.connector._                                                      
import org.apache.spark.sql.cassandra._ 
import com.datastax.spark.connector.cql.CassandraConnectorConf

In [10]:
val hostname = "cassandra"

Kubernetes/docker-compose are useful here, "cassandra" is the hostname of the cassandra-cluster

In [11]:
val conf = new SparkConf().setAppName("SparkApp")
conf.set("spark.driver.allowMultipleContexts", "true")
conf.set("spark.cassandra.connection.host", hostname)
val sc = new SparkContext(conf)

create some key/value pairs and insert them into cassandra if test.kv exists

In [3]:
val collection = sc.parallelize(Seq(("key3", 3), ("key4", 4))) 
var rdd = sc.cassandraTable("test", "kv")                                                          
collection.saveToCassandra("test", "kv", SomeColumns("key", "value")) 

Name: java.io.IOException
Message: Couldn't find test.kv or any similarly named keyspace and table pairs
StackTrace:   at com.datastax.spark.connector.cql.Schema$.tableFromCassandra(Schema.scala:356)
  at com.datastax.spark.connector.writer.TableWriter$.apply(TableWriter.scala:344)
  at com.datastax.spark.connector.RDDFunctions.saveToCassandra(RDDFunctions.scala:35)

In [12]:
val spark = SparkSession.builder.appName("Simple Application").getOrCreate()
spark.setCassandraConf("cluster", CassandraConnectorConf.ConnectionHostParam.option(hostname))

org.apache.spark.sql.SparkSession@df698a7

This displays the table if it exists (max 20)

In [18]:
val df = spark.read.format("org.apache.spark.sql.cassandra").options(Map(                                                                          
       "table" -> "kv",                                                               
       "keyspace" -> "test",
       "cluster" -> "cluster")).load()                                                         
      df.show()                                                                   

Name: java.io.IOException
Message: Couldn't find test.kv or any similarly named keyspace and table pairs
StackTrace:   at com.datastax.spark.connector.cql.Schema$.tableFromCassandra(Schema.scala:356)
  at org.apache.spark.sql.cassandra.CassandraSourceRelation.<init>(CassandraSourceRelation.scala:46)
  at org.apache.spark.sql.cassandra.CassandraSourceRelation$.apply(CassandraSourceRelation.scala:287)
  at org.apache.spark.sql.cassandra.DefaultSource.createRelation(DefaultSource.scala:56)
  at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:306)
  at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:178)
  at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:146)

In [17]:
sc.stop()
spark.stop()